# 비트코인과 금리를 일봉으로 비교해보자 

## 목적 : 월봉으로 비교했을때 비트코인과 금리는 어느정도 영향이 있는 것으로 확인되고 단기적으로 일봉으로도 과연 영향이 있을까? 해서 비교

In [71]:
import pandas as pd
import numpy as np

In [87]:
#비트코인 일봉 데려오기
bitcoin = pd.read_csv('../upbit/data/bitcoin.csv', encoding= 'utf-8')
bitcoin

,date,open,high,low,close,volume
0,2017-09-25,4201000.0,4333000.0,4175000.0,4322000.0,132.484755
1,2017-09-26,4317000.0,4418000.0,4311000.0,4321000.0,22.788340
2,2017-09-27,4322000.0,4677000.0,4318000.0,4657000.0,32.269662
3,2017-09-28,4657000.0,4772000.0,4519000.0,4586000.0,80.588243
4,2017-09-29,4586000.0,4709000.0,4476000.0,4657000.0,59.352373
...,...,...,...,...,...,...
1342,2021-05-29,43723000.0,44812000.0,41405000.0,42688000.0,10628.404447
1343,2021-05-30,42688000.0,43827000.0,40580000.0,43177000.0,9194.807917
1344,2021-05-31,43177000.0,44235000.0,41304000.0,43761000.0,10111.066220
1345,2021-06-01,43761000.0,44550000.0,42515000.0,43200000.0,9081.150288


# 시장금리와 비트코인 데이터 합치기

In [88]:
#시장금리 일별 데이터 가져오기 
interest_day = pd.read_csv('../upbit/output/시장금리_일별.csv', encoding= 'utf-8', index_col=0)
interest_day.isnull().sum()

날짜             0
콜금리            0
CD(91일)        0
국고채(3년)        0
국고채(5년)        0
회사채(3년,AA-)    0
dtype: int64

In [89]:
interest_day['국고채(5년)'].unique() # '-' 값 확인


array(['1.95', '1.96', '1.99', '1.98', '1.97', '2.02', '2.09', '2.14',
       '2.11', '2.12', '2.13', '2.15', '2.21', '2.3', '2.27', '2.29',
       '2.34', '2.42', '2.41', '2.39', '2.4', '2.37', '2.35', '2.36',
       '2.38', '2.31', '2.26', '2.28', '2.67', '2.33', '2.45', '2.43',
       '2.48', '2.47', '2.44', '2.46', '2.49', '2.56', '2.59', '2.57',
       '2.54', '2.5', '2.52', '2.53', '2.51', '2.55', '1.39', '2.58',
       '2.32', '2.18', '2.17', '2.16', '2.1', '2.08', '2.06', '2.23',
       '2.22', '2.19', '2.2', '2.24', '2.25', '2', '2.04', '2.07', '2.05',
       '2.01', '1.94', '1.88', '1.89', '1.87', '1.91', '1.9', '1.86',
       '1.83', '1.84', '1.85', '1.79', '1.74', '1.7', '1.71', '1.73',
       '1.75', '1.78', '1.76', '1.77', '1.81', '1.82', '-'], dtype=object)

In [90]:
interest_day.replace('-', 0, inplace=True) # '-' --> 0 으로 

interest_day['국고채(5년)'] = pd.to_numeric(interest_day['국고채(5년)'])
interest_day['국고채(3년)'] = pd.to_numeric(interest_day['국고채(3년)'])
interest_day['CD(91일)'] = pd.to_numeric(interest_day['CD(91일)'])

interest_day.rename(columns = {'날짜': 'date',
                               '콜금리' : 'call',
                               'CD(91일)' : 'CD91',
                               '국고채(3년)' : 'three',
                               '국고채(5년)' : 'five',
                               '회사채(3년,AA-)' : 'company_3'}, inplace=True)
interest_day


,date,call,CD91,three,five,company_3
0,2017.09.11,1.25,1.38,1.76,1.95,2.3
1,2017.09.12,1.25,1.38,1.75,1.96,2.3
2,2017.09.13,1.26,1.38,1.74,1.95,2.29
3,2017.09.14,1.26,1.38,1.74,1.95,2.29
4,2017.09.15,1.26,1.38,1.74,1.95,2.29
...,...,...,...,...,...,...
910,2021.05.26,0.47,0.66,1.16,0.00,1.9
911,2021.05.27,0.49,0.66,1.12,0.00,1.87
912,2021.05.28,0.43,0.66,1.16,0.00,1.9
913,2021.05.31,0.48,0.66,1.23,0.00,1.95


In [91]:
interest_day[interest_day['five'] != 0 ]

,date,call,CD91,three,five,company_3
0,2017.09.11,1.25,1.38,1.76,1.95,2.3
1,2017.09.12,1.25,1.38,1.75,1.96,2.3
2,2017.09.13,1.26,1.38,1.74,1.95,2.29
3,2017.09.14,1.26,1.38,1.74,1.95,2.29
4,2017.09.15,1.26,1.38,1.74,1.95,2.29
...,...,...,...,...,...,...
383,2019.04.11,1.75,1.86,1.73,1.76,2.2
384,2019.04.12,1.78,1.86,1.73,1.77,1.97
385,2019.04.15,1.76,1.86,1.76,1.81,2.23
386,2019.04.16,1.77,1.86,1.77,1.81,2.23


In [92]:
import numpy as np
interest_day.five.replace(0,np.nan,inplace=True)
interest_day['tmp'] = round(interest_day[['CD91','three','five']].mean(axis=1), 2)
interest_day

,date,call,CD91,three,five,company_3,tmp
0,2017.09.11,1.25,1.38,1.76,1.95,2.3,1.70
1,2017.09.12,1.25,1.38,1.75,1.96,2.3,1.70
2,2017.09.13,1.26,1.38,1.74,1.95,2.29,1.69
3,2017.09.14,1.26,1.38,1.74,1.95,2.29,1.69
4,2017.09.15,1.26,1.38,1.74,1.95,2.29,1.69
...,...,...,...,...,...,...,...
910,2021.05.26,0.47,0.66,1.16,NaN,1.9,0.91
911,2021.05.27,0.49,0.66,1.12,NaN,1.87,0.89
912,2021.05.28,0.43,0.66,1.16,NaN,1.9,0.91
913,2021.05.31,0.48,0.66,1.23,NaN,1.95,0.94


In [93]:
interest = interest_day[['date', 'tmp']]

In [94]:
interest['date'] = pd.to_datetime(interest['date'], format="%Y-%m")
interest['date'] = pd.to_datetime(interest['date']).dt.normalize()
print(interest)

          date   tmp
0   2017-09-11  1.70
1   2017-09-12  1.70
2   2017-09-13  1.69
3   2017-09-14  1.69
4   2017-09-15  1.69
..         ...   ...
910 2021-05-26  0.91
911 2021-05-27  0.89
912 2021-05-28  0.91
913 2021-05-31  0.94
914 2021-06-01  0.94

[915 rows x 2 columns]


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [95]:
interest.info()
print('-----')
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915 entries, 0 to 914
Data columns (total 2 columns):
date    915 non-null datetime64[ns]
tmp     915 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 21.4 KB
-----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1347 entries, 0 to 1346
Data columns (total 6 columns):
date      1347 non-null object
open      1347 non-null float64
high      1347 non-null float64
low       1347 non-null float64
close     1347 non-null float64
volume    1347 non-null float64
dtypes: float64(5), object(1)
memory usage: 63.3+ KB


In [96]:
bitcoin['date'] = pd.to_datetime(bitcoin['date'], format="%Y-%m")
bitcoin['date'] = pd.to_datetime(bitcoin['date']).dt.normalize()
print(bitcoin)

           date        open        high         low       close        volume
0    2017-09-25   4201000.0   4333000.0   4175000.0   4322000.0    132.484755
1    2017-09-26   4317000.0   4418000.0   4311000.0   4321000.0     22.788340
2    2017-09-27   4322000.0   4677000.0   4318000.0   4657000.0     32.269662
3    2017-09-28   4657000.0   4772000.0   4519000.0   4586000.0     80.588243
4    2017-09-29   4586000.0   4709000.0   4476000.0   4657000.0     59.352373
...         ...         ...         ...         ...         ...           ...
1342 2021-05-29  43723000.0  44812000.0  41405000.0  42688000.0  10628.404447
1343 2021-05-30  42688000.0  43827000.0  40580000.0  43177000.0   9194.807917
1344 2021-05-31  43177000.0  44235000.0  41304000.0  43761000.0  10111.066220
1345 2021-06-01  43761000.0  44550000.0  42515000.0  43200000.0   9081.150288
1346 2021-06-02  43207000.0  43435000.0  42804000.0  42865000.0    330.278103

[1347 rows x 6 columns]


In [97]:
result = pd.merge(bitcoin, interest, on= 'date', how="outer") 
result.tail(20)

,date,open,high,low,close,volume,tmp
1337,2021-05-24,42585000.0,47452000.0,41253000.0,46963000.0,22353.309349,0.89
1338,2021-05-25,46932000.0,49113000.0,45197000.0,47047000.0,15379.795809,0.90
1339,2021-05-26,47067000.0,49081000.0,46265000.0,47325000.0,11383.195814,0.91
1340,2021-05-27,47324000.0,48120000.0,45423000.0,46563000.0,10358.068049,0.89
1341,2021-05-28,46568000.0,47069000.0,43009000.0,43723000.0,14678.133632,0.91
1342,2021-05-29,43723000.0,44812000.0,41405000.0,42688000.0,10628.404447,NaN
1343,2021-05-30,42688000.0,43827000.0,40580000.0,43177000.0,9194.807917,NaN
1344,2021-05-31,43177000.0,44235000.0,41304000.0,43761000.0,10111.066220,0.94
1345,2021-06-01,43761000.0,44550000.0,42515000.0,43200000.0,9081.150288,0.94
1346,2021-06-02,43207000.0,43435000.0,42804000.0,42865000.0,330.278103,NaN


In [98]:
final = result.drop(index=[1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356], axis=0)
final.tail(20)

,date,open,high,low,close,volume,tmp
1327,2021-05-14,63080000.0,63656000.0,61001000.0,61797000.0,13221.575038,0.90
1328,2021-05-15,61795000.0,62118000.0,57900000.0,58807000.0,17507.507031,NaN
1329,2021-05-16,58814000.0,60721000.0,55500000.0,57649000.0,16552.587122,NaN
1330,2021-05-17,57655000.0,58000000.0,51381000.0,54057000.0,27456.362669,0.89
1331,2021-05-18,54057000.0,56366000.0,53001000.0,53419000.0,15075.110283,0.88
1332,2021-05-19,53404000.0,53992000.0,42595000.0,50379000.0,40112.808728,NaN
1333,2021-05-20,50477000.0,53070000.0,48328000.0,51320000.0,21350.740777,0.88
1334,2021-05-21,51318000.0,52726000.0,45701000.0,49112000.0,22198.519010,0.88
1335,2021-05-22,49080000.0,49260000.0,43957000.0,47460000.0,16597.850631,NaN
1336,2021-05-23,47444000.0,48592000.0,39331000.0,42585000.0,27384.564127,NaN


In [99]:
final.to_csv('../upbit/output/비트코인+일별시장금리.csv', encoding='utf-8')